<a href="https://colab.research.google.com/github/DATAITINA/my-ai-journey/blob/main/LangChain_RAG_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/Datasets/nigerian_food_database.csv'
data = pd.read_csv(data_path)
data

,food_id,name,portion_g,calories,protein_g,carbs_g,fat_g
0,NG001,Jollof Rice,250,400,7,70,10
1,NG002,Egusi Soup,200,350,15,12,28
2,NG003,Efo Riro,200,280,10,14,20
3,NG004,Pounded Yam,300,500,6,120,1
4,NG005,Moi Moi,150,250,12,20,10
5,NG006,Suya,100,300,25,5,20
6,NG007,Akara,100,280,10,22,18
7,NG008,Fried Plantain (Dodo),150,320,2,60,10
8,NG009,Pepper Soup,250,180,20,5,7
9,NG010,Okra Soup,200,220,9,15,12


In [ ]:
from langchain.docstore.document import Document

# Converting CSV rows
docs = []
for _, row in data.iterrows():
    text = (
        f"Food: {row['name']}, Portion: {row['portion_g']}g, "
        f"Calories: {row['calories']} kcal, Protein: {row['protein_g']}g, "
        f"Carbs: {row['carbs_g']}g, Fat: {row['fat_g']}g"
    )
    docs.append(Document(page_content=text))

In [ ]:
%pip install --upgrade --quiet langchain-google-genai
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.5 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from google.colab import userdata
import google.generativeai as genai

# Get the API key from Colab secrets and configure the library
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


# Using Gemini embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

db = FAISS.from_documents(docs, embeddings)

db.save_local("food_db")

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7f01c1b1a7b0>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 126, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/credentials.py", line 99, in _retrieve_info
    info = _metadata.get_service_account_info(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 338, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/auth/compute_engine/_metadata.py", line 263, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http:/

GoogleGenerativeAIError: Error embedding content: Timeout of 60.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f01c1a33bc0>)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature = 0)
retriever = db.load_local("food_db", embeddings, allow_dangerous_deserialization=True).as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
query = "From the dataset what are the nutrients in akara? Give a detailed explanation, generate examples and provide other foods you think have close nutrients to akara.."
response = qa.invoke(query)
print(response['result'])

Based on the provided data, 100g of Akara contains:

* **Calories:** 280 kcal
* **Protein:** 10g
* **Carbohydrates:** 22g
* **Fat:** 18g

This shows that Akara is relatively high in calories and fat, with a moderate amount of protein and carbohydrates.  It's a good source of energy, but also relatively high in fat compared to some other foods.

I don't have enough information to suggest other foods with *close* nutrient profiles to Akara.  The dataset only provides information on four specific foods, and doesn't offer a broader comparison across a wider range of foods.  To find similar foods, a more extensive nutritional database would be needed.
